In [1]:
from __future__ import print_function
from matplotlib import pyplot as plt
# get_ipython().magic(u'matplotlib auto')
import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import torch
from tensorflow import keras
import statistics

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow import reshape
from keras.utils import np_utils
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D,AveragePooling1D
from tensorflow.keras.layers import SeparableConv1D, Bidirectional
from tensorflow.keras.layers import LocallyConnected2D
from tensorflow.keras.layers import ZeroPadding2D,ZeroPadding1D, MaxPooling2D, Bidirectional
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import CSVLogger,LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [3]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from numpy import std
# import coremltools
# from torch import nn, optim
# import torch.nn.functional as F
import scipy.io
from scipy import stats
#from IPython.display import display, HTML
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import TimeDistributed
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [4]:
# The number of steps within one time segment
TIME_PERIODS = 100
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 100
N_FEATURES = 12

In [5]:
column_names = ['C'+str(j) for j in range(1, N_FEATURES+1)]
lst = ['Class_label']
column_names = column_names+lst
print(column_names)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'Class_label']


In [6]:
def read_data_Train(file_path_Train):
    df_Train = pd.read_csv(file_path_Train,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Train['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Train['Class_label'] = df_Train['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Train.dropna(axis=0, how='any', inplace=True)
    return df_Train
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

In [7]:
def show_basic_dataframe_info(dataframe):
    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

In [8]:
def read_data_Test(file_path_Test):
    df_Test = pd.read_csv(file_path_Test,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Test['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Test['Class_label'] = df_Test['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Test.dropna(axis=0, how='any', inplace=True)
    return df_Test

In [9]:
 pd.options.display.float_format = "{:,.5f}".format

In [10]:
# 15 function to segment data into trial lengths (trial length =513 samples in this dataset)
def create_segments_and_labels(df, time_steps,step,n_features, label_name):
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
      for j in range(1, n_features+1):
        L = ('C'+str(j)) 
        segments.append(df[str(L)].values[i: i + time_steps])
      label = stats.mode(df[label_name][i: i + time_steps])[0][0]
      labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [11]:
verbose, epochs, batch_size = 0, 160, 512

In [12]:
# def step_decay(epoch):
#    initial_lrate = 0.1
#    drop = 0.1
#    epochs_drop = 16.0
#    lrate = initial_lrate * tf.math.pow(drop,  
#            tf.math.floor((1+epoch)/epochs_drop))
#    return lrate

In [13]:
def step_decay(epoch):
   initial_lrate = 1e-3
   drop = 0.1
   epochs_drop = 40.0
   lrate = initial_lrate * tf.math.pow(drop,  
           tf.math.floor((1+epoch)/epochs_drop))
   return lrate

In [14]:
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)

In [15]:
# lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
test_acc=[]

In [16]:
# import os
path='/media/naveen/nav/mat_codes/nina_DB4_codes/olsson_prep_nb/sub_wise_process_TT/'

In [17]:
for i in range(1,11):
    file_path_train=path+'S'+str(i)+'_'+'tr'+'.'+'csv'
    print(file_path_train)
    df_Train=read_data_Train(file_path_train)
    show_basic_dataframe_info(df_Train)
    df_Train.head(5)
    file_path_test=path+'S'+str(i)+'_'+'tt'+'.'+'csv'
    print(file_path_test)
    df_Test=read_data_Test(file_path_test)
    show_basic_dataframe_info(df_Test)
    df_Test.head(5)
    
    scaler = preprocessing.StandardScaler().fit(df_Train.iloc[:,0:N_FEATURES])
    df_Train.iloc[:,0:N_FEATURES]=scaler.transform(df_Train.iloc[:,0:N_FEATURES])
    df_Test.iloc[:,0:N_FEATURES]=scaler.transform(df_Test.iloc[:,0:N_FEATURES])
       
    LABEL = 'ActivityEncoded'
    # Transform the labels from String to Integer via LabelEncoder
    le = preprocessing.LabelEncoder()
    # Add a new column to the existing DataFrame with the encoded values
    df_Train[LABEL] = le.fit_transform(df_Train['Class_label'].values.ravel())
    # df_Valid[LABEL] = le.fit_transform(df_Valid['Class_label'].values.ravel())
    df_Test[LABEL] = le.fit_transform(df_Test['Class_label'].values.ravel())
    x_train, y_train = create_segments_and_labels(df_Train,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_train shape: ', x_train.shape)
    # print(x_train)
    print(x_train.shape[0], 'training samples')
    print('y_train shape: ', y_train.shape)
    # Set input & output dimensions
    num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
    print('num_time_periods',num_time_periods)
    print('num_sensors',num_sensors)
    num_classes = le.classes_.size
    print('class_list',list(le.classes_))
    # input_shape = (num_time_periods,num_sensors)
    # print(input_shape)
    input_shape = (num_time_periods,num_sensors)
    #x_train = x_train.reshape(x_train.shape[0], input_shape)
    print('x_train shape:', x_train[0].shape)
    print('input_shape:', input_shape)
    x_train = x_train.astype('float32')
    # x_train = [torch.tensor(arr, dtype=torch.float32) for arr in x_train]
    # y_train = y_train.astype('float32')
    print('y_train shape',y_train.shape)
    y_train_hot = np_utils.to_categorical(y_train, num_classes)
#     print(y_train_hot)
    # y_train_hot= [torch.tensor(arr, dtype=torch.uint8) for arr in y_train_hot]
    print('New y_train shape: ', y_train_hot.shape)
    x_test, y_test = create_segments_and_labels(df_Test,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_test shape: ', x_test.shape)
    # print(x_train)
    print(x_test.shape[0], 'testing samples')
    print('y_test shape: ', y_test.shape)
    # Set input_shape / reshape for Keras
    #x_test = x_test.reshape(x_test.shape[0], input_shape)
    x_test = x_test.astype('float32')
    y_test = y_test.astype('float32')
    y_test_hot = np_utils.to_categorical(y_test, num_classes)
    # n_steps, n_length = 20, 25
    # n_steps, n_length= 10, 50
    # n_steps, n_length= 16, 32
    n_steps, n_length = 100, 12
    n_depth=1
    x_train = x_train.reshape(x_train.shape[0], n_steps, n_length,n_depth)
    print('x_train shape: ', x_train.shape)
    # x_valid = x_valid.reshape(x_valid.shape[0], n_steps, n_length, n_depth)
    # print('x_valid shape: ', x_valid.shape)
    x_test = x_test.reshape(x_test.shape[0], n_steps, n_length,n_depth)
    print('x_test shape: ', x_test.shape)
    n_outputs = y_train_hot.shape[1]
    print('n_outputs',n_outputs)
#     model = Sequential()
#     model.add(Conv2D(filters=64, kernel_size=(9,1), padding='same',kernel_regularizer=l2(1e-04), kernel_initializer="he_normal",strides=(3,1), \
#                            input_shape=(n_steps,n_length,n_depth)))
#     model.add(MaxPooling2D(pool_size=(8,1),strides=(2,1)))
#     model.add(Activation('tanh'))
#     model.add(Conv2D(filters=64, kernel_size=(5,1),padding="same",kernel_regularizer=l2(1e-04),kernel_initializer="he_normal",strides=(2,1)))
#     model.add(Activation('tanh'))
#     model.add(Dropout(0.2093))
#     model.add(Activation('relu'))
#     model.add(Flatten())
#     model.add(Dense(512, activation='tanh'))
#     model.add(Dense(n_outputs, activation='softmax'))
#     adam=optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999,epsilon=1e-08, amsgrad=False)
#     sgd=optimizers.SGD(learning_rate=2.17e-3, momentum=0.9, nesterov=False, name='SGD')
    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size=(9,1), padding='same', kernel_initializer="he_normal",strides=(3,1),kernel_regularizer=l1(1e-04), \
                           input_shape=(n_steps,n_length,n_depth)))
    model.add(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None))
    model.add(MaxPooling2D(pool_size=(8,1),strides=(2,1)))
    model.add(Activation('tanh'))
    model.add(Conv2D(filters=64, kernel_size=(5,1),padding="same",kernel_initializer="he_normal",strides=(2,1),kernel_regularizer=l1(1e-04)))
    model.add(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None))
    model.add(Activation('tanh'))
    model.add(Dropout(0.2093))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='tanh',kernel_initializer="he_normal"))
    model.add(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None))
    model.add(Activation('tanh'))
    model.add(Dense(n_outputs, activation='softmax',kernel_initializer="he_normal"))
    adam=optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999,epsilon=1e-08, amsgrad=False)
    checkpoint_filepath = '/media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/checkpoint.hdf5'
    # model.load_weights(checkpoint_filepath) 
    checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,verbose=1, monitor='val_accuracy',save_weights_only=True,save_best_only=True)
    early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
    print(model.summary())
#     tf.keras.utils.plot_model(model, to_file='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/Model1.png',show_shapes=True,show_layer_names=True,dpi=96)
    csv_logger = CSVLogger('/media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/LSTM_nina_20X100.csv', append=True, separator=';')
    history = model.fit(x_train, y_train_hot, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger,checkpoint_callback,lrate,early],validation_data=(x_test, y_test_hot), verbose=1)
    best_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
    print('epoch_number',best_index+1)
    print('train accuracy and validation accuracy', history.history['accuracy'][best_index], history.history['val_accuracy'][best_index])
    model.load_weights(checkpoint_filepath) 
    _, testaccuracy = model.evaluate(x_test, y_test_hot, batch_size=batch_size, verbose=1)
    print('test_accuracy',testaccuracy)
    test_acc.append(testaccuracy)
    print(test_acc)
    print('test_mean for %d subjects:',i)
#     print(test_acc)
    test_mean=statistics.mean(test_acc)
    print(test_mean)

/media/naveen/nav/mat_codes/nina_DB4_codes/olsson_prep_nb/sub_wise_process_TT/S1_tr.csv
Number of columns in the dataframe: 13
Number of rows in the dataframe: 2059200

/media/naveen/nav/mat_codes/nina_DB4_codes/olsson_prep_nb/sub_wise_process_TT/S1_tt.csv
Number of columns in the dataframe: 13
Number of rows in the dataframe: 1029600

x_train shape:  (20591, 100, 12)
20591 training samples
y_train shape:  (20591,)
num_time_periods 100
num_sensors 12
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0]
x_train shape: (100, 12)
input_shape: (100, 12)
y_train shape (20591,)
New y_train shape:  (20591, 52)
x_test shape:  (10295, 100, 12)
10295 testing samples
y_test shape:  (10295,)
x_train shape:  (20591, 100, 12, 1)
x_test shape:

Epoch 16/160
40/41 [============================>.] - ETA: 0s - loss: 0.2411 - accuracy: 0.9909
Epoch 00016: val_accuracy did not improve from 0.14900
41/41 [==============================] - 1s 18ms/step - loss: 0.2412 - accuracy: 0.9909 - val_loss: 4.6578 - val_accuracy: 0.1361 - lr: 0.0010
Epoch 17/160
40/41 [============================>.] - ETA: 0s - loss: 0.2220 - accuracy: 0.9942
Epoch 00017: val_accuracy did not improve from 0.14900
41/41 [==============================] - 1s 21ms/step - loss: 0.2220 - accuracy: 0.9942 - val_loss: 4.7554 - val_accuracy: 0.1404 - lr: 0.0010
Epoch 18/160
39/41 [===========================>..] - ETA: 0s - loss: 0.2114 - accuracy: 0.9943
Epoch 00018: val_accuracy did not improve from 0.14900
41/41 [==============================] - 1s 20ms/step - loss: 0.2114 - accuracy: 0.9944 - val_loss: 4.7620 - val_accuracy: 0.1461 - lr: 0.0010
Epoch 19/160
40/41 [============================>.] - ETA: 0s - loss: 0.2008 - accuracy: 0.9956
Epoch 00019: val_accur

Epoch 44/160
36/41 [=========================>....] - ETA: 0s - loss: 0.1298 - accuracy: 0.9995
Epoch 00044: val_accuracy did not improve from 0.15231
41/41 [==============================] - 1s 17ms/step - loss: 0.1298 - accuracy: 0.9995 - val_loss: 5.2359 - val_accuracy: 0.1512 - lr: 1.0000e-04
Epoch 45/160
41/41 [==============================] - ETA: 0s - loss: 0.1286 - accuracy: 0.9991
Epoch 00045: val_accuracy did not improve from 0.15231
41/41 [==============================] - 1s 17ms/step - loss: 0.1286 - accuracy: 0.9991 - val_loss: 5.2357 - val_accuracy: 0.1502 - lr: 1.0000e-04
Epoch 46/160
37/41 [==========================>...] - ETA: 0s - loss: 0.1277 - accuracy: 0.9993
Epoch 00046: val_accuracy did not improve from 0.15231
41/41 [==============================] - 1s 19ms/step - loss: 0.1278 - accuracy: 0.9993 - val_loss: 5.2400 - val_accuracy: 0.1513 - lr: 1.0000e-04
Epoch 47/160
39/41 [===========================>..] - ETA: 0s - loss: 0.1279 - accuracy: 0.9992
Epoch 0004

Epoch 71/160
39/41 [===========================>..] - ETA: 0s - loss: 0.1169 - accuracy: 0.9998
Epoch 00071: val_accuracy improved from 0.15454 to 0.15474, saving model to /media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/checkpoint.hdf5
41/41 [==============================] - 1s 19ms/step - loss: 0.1170 - accuracy: 0.9997 - val_loss: 5.3234 - val_accuracy: 0.1547 - lr: 1.0000e-04
Epoch 72/160
36/41 [=========================>....] - ETA: 0s - loss: 0.1166 - accuracy: 0.9997
Epoch 00072: val_accuracy did not improve from 0.15474
41/41 [==============================] - 1s 18ms/step - loss: 0.1166 - accuracy: 0.9997 - val_loss: 5.3161 - val_accuracy: 0.1487 - lr: 1.0000e-04
Epoch 73/160
40/41 [============================>.] - ETA: 0s - loss: 0.1159 - accuracy: 0.9997
Epoch 00073: val_accuracy did not improve from 0.15474
41/41 [==============================] - 1s 18ms/step - loss: 0.1159 - accuracy: 0.9997 - val_loss: 5.3120 - val_accuracy: 0.1514 - lr: 1.0000e-04
Epoch

37/41 [==========================>...] - ETA: 0s - loss: 0.1128 - accuracy: 0.9998
Epoch 00098: val_accuracy did not improve from 0.15474
41/41 [==============================] - 1s 17ms/step - loss: 0.1128 - accuracy: 0.9998 - val_loss: 5.3307 - val_accuracy: 0.1519 - lr: 1.0000e-05
Epoch 99/160
41/41 [==============================] - ETA: 0s - loss: 0.1123 - accuracy: 0.9998
Epoch 00099: val_accuracy did not improve from 0.15474
41/41 [==============================] - 1s 18ms/step - loss: 0.1123 - accuracy: 0.9998 - val_loss: 5.3298 - val_accuracy: 0.1517 - lr: 1.0000e-05
Epoch 100/160
40/41 [============================>.] - ETA: 0s - loss: 0.1119 - accuracy: 0.9999
Epoch 00100: val_accuracy did not improve from 0.15474
41/41 [==============================] - 1s 18ms/step - loss: 0.1119 - accuracy: 0.9999 - val_loss: 5.3322 - val_accuracy: 0.1516 - lr: 1.0000e-05
Epoch 101/160
40/41 [============================>.] - ETA: 0s - loss: 0.1125 - accuracy: 0.9997
Epoch 00101: val_accu

...

41/41 [==============================] - ETA: 0s - loss: 0.1014 - accuracy: 0.9999
Epoch 00081: val_accuracy did not improve from 0.22399
41/41 [==============================] - 1s 17ms/step - loss: 0.1014 - accuracy: 0.9999 - val_loss: 4.3530 - val_accuracy: 0.2202 - lr: 1.0000e-05
Epoch 82/160
36/41 [=========================>....] - ETA: 0s - loss: 0.1011 - accuracy: 0.9997
Epoch 00082: val_accuracy did not improve from 0.22399
41/41 [==============================] - 1s 15ms/step - loss: 0.1011 - accuracy: 0.9998 - val_loss: 4.3538 - val_accuracy: 0.2208 - lr: 1.0000e-05
Epoch 83/160
36/41 [=========================>....] - ETA: 0s - loss: 0.1010 - accuracy: 1.0000
Epoch 00083: val_accuracy did not improve from 0.22399
41/41 [==============================] - 1s 17ms/step - loss: 0.1010 - accuracy: 1.0000 - val_loss: 4.3501 - val_accuracy: 0.2208 - lr: 1.0000e-05
Epoch 84/160
40/41 [============================>.] - ETA: 0s - loss: 0.1007 - accuracy: 1.0000
Epoch 00084: val_accura

Epoch 1/160
40/41 [============================>.] - ETA: 0s - loss: 3.5357 - accuracy: 0.1288
Epoch 00001: val_accuracy improved from -inf to 0.11083, saving model to /media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/checkpoint.hdf5
41/41 [==============================] - 1s 21ms/step - loss: 3.5341 - accuracy: 0.1288 - val_loss: 3.4900 - val_accuracy: 0.1108 - lr: 0.0010
Epoch 2/160
36/41 [=========================>....] - ETA: 0s - loss: 2.8036 - accuracy: 0.2820
Epoch 00002: val_accuracy improved from 0.11083 to 0.14803, saving model to /media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/checkpoint.hdf5
41/41 [==============================] - 1s 17ms/step - loss: 2.7914 - accuracy: 0.2847 - val_loss: 3.3151 - val_accuracy: 0.1480 - lr: 0.0010
Epoch 3/160
37/41 [==========================>...] - ETA: 0s - loss: 2.3696 - accuracy: 0.3992
Epoch 00003: val_accuracy improved from 0.14803 to 0.17542, saving model to /media/naveen/nav/mat_codes/nina_DB2_codes/o

Epoch 26/160
41/41 [==============================] - ETA: 0s - loss: 0.1633 - accuracy: 0.9977
Epoch 00026: val_accuracy did not improve from 0.18776
41/41 [==============================] - 1s 16ms/step - loss: 0.1633 - accuracy: 0.9977 - val_loss: 4.4547 - val_accuracy: 0.1857 - lr: 0.0010
Epoch 27/160
36/41 [=========================>....] - ETA: 0s - loss: 0.1586 - accuracy: 0.9978
Epoch 00027: val_accuracy did not improve from 0.18776
41/41 [==============================] - 1s 16ms/step - loss: 0.1586 - accuracy: 0.9978 - val_loss: 4.5020 - val_accuracy: 0.1874 - lr: 0.0010
Epoch 28/160
41/41 [==============================] - ETA: 0s - loss: 0.1546 - accuracy: 0.9984
Epoch 00028: val_accuracy did not improve from 0.18776
41/41 [==============================] - 1s 16ms/step - loss: 0.1546 - accuracy: 0.9984 - val_loss: 4.5799 - val_accuracy: 0.1854 - lr: 0.0010
Epoch 29/160
41/41 [==============================] - ETA: 0s - loss: 0.1559 - accuracy: 0.9976
Epoch 00029: val_accur

37/41 [==========================>...] - ETA: 0s - loss: 0.1193 - accuracy: 0.9999
Epoch 00053: val_accuracy did not improve from 0.19767
41/41 [==============================] - 1s 18ms/step - loss: 0.1195 - accuracy: 0.9999 - val_loss: 4.6977 - val_accuracy: 0.1976 - lr: 1.0000e-04
Epoch 54/160
37/41 [==========================>...] - ETA: 0s - loss: 0.1195 - accuracy: 0.9997
Epoch 00054: val_accuracy improved from 0.19767 to 0.19835, saving model to /media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/checkpoint.hdf5
41/41 [==============================] - 1s 20ms/step - loss: 0.1195 - accuracy: 0.9998 - val_loss: 4.7200 - val_accuracy: 0.1983 - lr: 1.0000e-04
Epoch 55/160
39/41 [===========================>..] - ETA: 0s - loss: 0.1194 - accuracy: 0.9996
Epoch 00055: val_accuracy improved from 0.19835 to 0.19854, saving model to /media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/checkpoint.hdf5
41/41 [==============================] - 1s 18ms/step - loss: 0.

Epoch 79/160
41/41 [==============================] - ETA: 0s - loss: 0.1112 - accuracy: 0.9998
Epoch 00079: val_accuracy did not improve from 0.20107
41/41 [==============================] - 1s 16ms/step - loss: 0.1112 - accuracy: 0.9998 - val_loss: 4.7522 - val_accuracy: 0.1983 - lr: 1.0000e-04
Epoch 80/160
36/41 [=========================>....] - ETA: 0s - loss: 0.1105 - accuracy: 0.9999
Epoch 00080: val_accuracy did not improve from 0.20107
41/41 [==============================] - 1s 16ms/step - loss: 0.1106 - accuracy: 0.9999 - val_loss: 4.7508 - val_accuracy: 0.2004 - lr: 1.0000e-05
Epoch 81/160
40/41 [============================>.] - ETA: 0s - loss: 0.1107 - accuracy: 0.9999
Epoch 00081: val_accuracy did not improve from 0.20107
41/41 [==============================] - 1s 16ms/step - loss: 0.1107 - accuracy: 0.9999 - val_loss: 4.7484 - val_accuracy: 0.2000 - lr: 1.0000e-05
Epoch 82/160
40/41 [============================>.] - ETA: 0s - loss: 0.1103 - accuracy: 1.0000
Epoch 0008

In [18]:
statistics.mean(test_acc)

0.22399222999811172